In [2]:
# Trials of objects for the robo platform
# HDF5 class for storing object parameters and provenance (not sure how to do the latter)

# regular imports
import numpy as np
from pathlib import Path

# imports of objects
# requires 'pip install cfunits', and 'conda install -c conda-forge udunits2' on the command line
from ObjectClasses.substrates import glassSlide
from ObjectClasses.racks import slideRackSven, gripper
from ObjectClasses.tables import opticalTableL, breadboard4545, breadboard1545, breadboard1515, breadboard1530, breadboard3045, breadboard1515C
from ObjectClasses.motionplatforms import roboArm
from ObjectClasses.cylinders import robotCylinder
from ObjectClasses.smtProfiles import smt_50x25
from ObjectClasses.syringeActuator import Alladin_1000
from ObjectClasses.breadBoardCube import simpleBreadBoardCube
from ObjectClasses.microscopes import wildType374547
from ObjectClasses.hotPlate import IKA_HotPlate
from RoboticSystemScene import RoboticSystemScene
import pythreejs
import ipywidgets
from IPython.display import HTML, display

# for 3D volume visualisation:
# install nodeJS LTS from nodejs website 
# conda install -c conda-forge ipyvolume
# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# jupyter labextension install ipyvolume
# jupyter labextension install jupyter-threejs
# pip install bqplot
# jupyter labextension install bqplot
import ipyvolume as ipv
# pip install vaex
# import vaex
# loading STL:
# pip install numpy-stl
from stl import mesh
# try using mplot3d for vis
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
# for rotations:
from scipy.spatial import transform
# for robot arm simulation:
# pip install git+https://github.com/Mecademic/python_driver
# import MecademicRobot
from operator import add

In [3]:
# yea, that doesn't work without an actual robot, not even in sim mode...
# robot = MecademicRobot.RobotController('192.168.0.100')

In [4]:
objectNames = []
# put all the objects in here, so we can process them all later. 
# let's start with a table:
mainTable= opticalTableL(
    filename = Path('mainTable.h5'), 
    deleteExisting = True, 
    name = 'main optical base table',
    rotation = [0., 0., 0.],
    location = [0., 0., 0.]
)

# We need the breadboard on which the SMT profile stands
smtBreadboard= breadboard1530(
    filename = Path('smtBreadboard.h5'), 
    deleteExisting = True, 
    name = 'breadboard on which SMT profile stands', 
    parent = mainTable, 
    rotation = [0., 0., -90.],
    location = list(map(add, mainTable.positions[29, 14], [12.5, 12.5, 0.]))
)


# I need a slide rack on a smaller breadboard:
rackBreadboard= breadboard3045(
    filename = Path('rackBreadboard.h5'), 
    deleteExisting = True, 
    name = 'breadboard on which the slide storage rack sits', 
    parent = mainTable, 
    rotation = [0., 0., 0.],
    location = list(map(add, mainTable.positions[5, 7], [0., 0., 17.]))
)

# put the robot arm breadboard on it (small, centered version), somewhere in the middle
armBreadboard= breadboard1530(
    filename = Path('armBreadboard.h5'), 
    deleteExisting = True, 
    name = 'breadboard on which the cylinders sit', 
    parent = mainTable, 
    rotation = [0, 0, 0],
    location = list(map(add, mainTable.positions[5, 19], [12.5, 12.5, 0.]))
)

# put the robot arm breadboard on it (small, centered version), somewhere in the middle
cameraBreadboard= breadboard1515(
    filename = Path('cameraBreadboard.h5'), 
    deleteExisting = True, 
    name = 'breadboard on which the camera is mounted', 
    parent = mainTable, 
    rotation = [0, 0, 0],
    location = list(map(add, mainTable.positions[24, 8], [12.5, 12.5, 0.]))        
)

# I need a slide rack on a smaller breadboard:
hotplateBreadboard= breadboard3045(
    filename = Path('HPBreadboard.h5'), 
    deleteExisting = True, 
    name = 'breadboard on which the hotplate sits', 
    parent = mainTable, 
    rotation = [0, 0, 0],
    location = list(map(add, mainTable.positions[2, 26], [12.5, 12.5, 0.]))
)

# We need the smt profile that holds Alladin
smtAlladin = smt_50x25(
    filename = Path('smtAlladin.h5'), 
    deleteExisting = True, 
    name = 'SMT profile that holds the Alladin syringe pump', 
    parent = smtBreadboard, 
    rotation = [0, 0, 0],
    location = list(map(add, smtBreadboard.positions[8, 0], [12.5, 17., 0.]))
)

# We need the smt profile that holds Alladin
alladinPump = Alladin_1000(
    filename = Path('Alladin.h5'), 
    deleteExisting = True, 
    name = 'Alladin syringe pump', 
    parent = smtAlladin, 
    rotation = [0, 0, 0],
    location = [0., 12.5, 566.]
)

armCylinders = []

# put the 4 cylinders on the robot arm breadboard
for i in range(2):
    for j in range(2):
        armCylinders += [robotCylinder(
            filename = Path('armCylinder{}.h5'.format(i*2 + j + 1)), 
            deleteExisting = True, 
            name = 'cylinder on which the robot arm sits', 
            parent = armBreadboard, 
            rotation = [0, 0, 0],
            location = armBreadboard.positions[6 + i*4, 1 + j*3]
        )]

# and the slide rack itself:
rack1 = slideRackSven(
    filename = Path('slideRack1.h5'), 
    stlFilename = Path('ObjectClasses/STLs/Slide_rack_text.stl'),
    name = 'Slide Rack a la Sven',
    parent = rackBreadboard,
    rotation = [0, 0, 0], # rotation vector / Euler vector, units in degrees
    location = list(map(add, rackBreadboard.positions[15, 9], [12.5, 0., 0.]))
)

# now I just need a slide or two:
slide1= glassSlide(
    filename = Path('testSlide.h5'), 
    deleteExisting = True, 
    name = 'my first glass slide', 
    stlFilename = Path('ObjectClasses/STLs/Glass_slide_text.stl'),
    location = rack1.positions['B'],
    parent = rack1
)


slide2= glassSlide(
    filename = Path('testSlide2.h5'), 
    extent = [-25, 25, -12, 12, 0, 0.17], 
    name = 'my second glass slide',
    location = rack1.positions['D'],
    rotation = [0, 0, 10],
    parent = rack1
)

# put the robot arm on it:
arm = roboArm(
    filename = Path('arm.h5'), 
    deleteExisting = True, 
    name = 'robot arm', 
    parent = armCylinders[0], 
    location = [50., 37.5, armCylinders[0].height],
    rotation = [0, 0, 0],
    positions = {'actuator': np.array([180., -60, -10.])}, # in external units (mm)
    orientations = {'actuator': [0, 0, 180]} # in external units (degree)
) # position of the robot arm (arm.positions['actuator']) would change over time, wonder if location of gripper can handle movements of position of attachment point?

#attach the gripper:
grip = gripper(
    filename = Path('grip.h5'), 
    deleteExisting = True, 
    name = 'gripper', 
    stlFilename = Path('ObjectClasses/STLs/doubleGrips.stl'),
    parent = arm, 
    location = arm.positions['actuator'],
    rotation = arm.orientations['actuator'], # same same
    # positions = {'actuator': [0, 66, -11]}
)

testCube = simpleBreadBoardCube(
    filename = Path('testCube.h5'), 
    deleteExisting = True, 
    name = 'test_cube',
    parent = rackBreadboard, 
    location = rackBreadboard.positions[5,5],
    rotation = [0., 0., 0.]
)

microscope = wildType374547(
    filename = Path('microscope.h5'),
    stlFilename = Path('ObjectClasses/STLs/Microscope.stl'),
    deleteExisting = True, 
    name = 'microscope',
    parent = mainTable, 
    location = list(map(add, mainTable.positions[24,29], [12.5, 10., 0.])),
    rotation = [0., 0., 0.]
)

hotplate = IKA_HotPlate(
    filename = Path('hotplate.h5'),
    stlFilename = Path('ObjectClasses/STLs/Hotplate.stl'),
    deleteExisting = True, 
    name = 'hotplate',
    parent = hotplateBreadboard, 
    location = list(map(add, hotplateBreadboard.positions[16, 1], [10., 3., 0.])),
    rotation = [0., 0., 0.]
)

breadBoardOnHotplate = breadboard1515C(
    filename = Path('breadBoardOnHotplate.h5'),
    deleteExisting = True, 
    name = 'breadboard on top of hotplate',
    parent = hotplate, 
    location = hotplate.positions[0],
    rotation = [0., 0., 0.]
)


# Collect the objects inside a list
objects = [
    mainTable,
    smtBreadboard,
    rackBreadboard,
    armBreadboard,
    cameraBreadboard,
    hotplateBreadboard,
    smtAlladin,
    alladinPump,
    rack1,
    slide1,
    slide2,
    arm,
    grip,
    testCube,
    microscope,
    hotplate,
    breadBoardOnHotplate] + armCylinders

# [setattr(key, obj) for key, obj in objects.items()]
# for obj in objectList:
#     obj.store() # not everything is stored such as parent and STL data. TODO: fix. it doesn't know how to deal with parent
# now everything should be initialized and I can visualise the set-up (maybe)

In [5]:
robot_sys_scene = RoboticSystemScene(objects=objects)
robot_sys_scene.convertObjectsToMeshes()
robot_sys_scene.createSceneThreeJs()
robot_sys_scene.showSceneThreeJs()

Renderer(camera=PerspectiveCamera(aspect=2.6666666666666665, position=(1000.0, 0.0, 1000.0), projectionMatrix=…

In [31]:
robot_sys_scene.translateObject(np.array([0.,0., 100.]), 0)

In [7]:
robot_sys_scene.scene_3js.children[2].position

(0.0, 0.0, 0.0)

In [8]:
robot_sys_scene.rotateObject(np.array([0.,0., 90.]), 0)

In [13]:
# move arm position up, see if the gripper inherits that move:
# put the robot arm on it:
arm.positions['actuator'] += [0, 0, 100] # in external units (mm)
arm.positions

{'actuator': array([180., -60., 190.])}

In [14]:
grip.location

array([180., -60., -10.])

In [15]:
# Matplotlib based visualization
def showObjects(objects):
    # Create a new plot
#     figure = ipv.figure()
    figure = plt.figure(figsize = [10, 8])
    axes = mplot3d.Axes3D(figure)

    # let's get some colors:
    rbColors = plt.cm.rainbow(np.linspace(0, 1, len(objects)))

    # Load the STL files and add the vectors to the plot
    for num, obj in enumerate(objects):
        print(f'working on object {obj.name}')
        if obj in [mainTable]:
            continue # don't show these, mplot3d isn't very good at it. 
        objStlVectors = obj.renderAbsoluteStlVectors()
        # we draw the tetrahedron
        # axes = ipv.plot_surface(objStlVectors[:,0], objStlVectors[:,1], objStlVectors[:,2], color=rbColors[num])
        axes.add_collection3d(mplot3d.art3d.Poly3DCollection(objStlVectors, color = rbColors[num]))

    # Auto scale to the mesh size
    scale = mainTable.stl.points.flatten()
    axes.set_xlim(250, 1000)
    axes.set_xlabel('x')
    axes.set_ylim(0, 750)
    axes.set_ylabel('y')
    axes.set_zlim(-250, 500)
    axes.set_zlabel('z')
    # axes.set_aspect(1)
    # doesn't work:
    # mplot3d.art3d.rotate_axes(0, 0, 90, 'z')
    # Show the plot to the screen
    plt.legend()
    plt.show()
    for obj in objects:
        print(f'{obj.name}\n \t extent: {obj.extent}, absLoc: {obj.getAbsoluteLocation()}')
# showObjects(objects)

In [27]:
# Animation track and' rotation setup

positon_track = pythreejs.VectorKeyframeTrack(name='.position',
    times=[0, 2, 5],
    values=[10, 6, 10,
            6.3, 3.78, 6.3,
            -2.98, 0.84, 9.2,
        ])
rotation_track = pythreejs.QuaternionKeyframeTrack(name='.quaternion',
    times=[0, 2, 5],
    values=[-0.184, 0.375, 0.0762, 0.905,
            -0.184, 0.375, 0.0762, 0.905,
            -0.0430, -0.156, -0.00681, 0.987,
        ])

camera_clip = pythreejs.AnimationClip(tracks=[positon_track, rotation_track])
camera_action = pythreejs.AnimationAction(pythreejs.AnimationMixer(camera)

SyntaxError: unexpected EOF while parsing (<ipython-input-27-b088133b3a01>, line 17)

In [ ]:
# for i in range(len(meshes)):
#     meshes[i].receiveShadow = True
#     meshes[i].castShadow = True

In [ ]:
# Visualize the objects' meshes
scene = showMeshesThreeJs(meshes, camera)

In [ ]:
animation

In [ ]:
def rotation_around_axis(axis, degrees, mesh):

    rotation_null = Quaternion(axis=axis, degrees=0.)
    rotation_null = list(rotation_null.elements)

    rotation = Quaternion(axis=axis, degrees=degrees)
    rotation = list(rotation.elements)

    rotation_null.extend(rotation)
    values = rotation_null

    track = QuaternionKeyframeTrack(name='.quaternion', times=[0,1], values=values)
    clip = AnimationClip(tracks=[track])
    animation = AnimationAction(mixer=AnimationMixer(), clip=clip, localRoot=mesh)

    return animation

In [ ]:
animation = rotation_around_axis([1,0,0], 90., meshes[1])

In [ ]:
meshes[8].position = (0., 0., 0.)

In [ ]:
meshes[8].quaternion = (0., 0., 0., 1.)

In [ ]:
meshes[8].rotateZ(1.)

In [ ]:
track1 = NumberKeyframeTrack(name='.rotation[z]', times=[0,1], values=[0,1])
clip1 = AnimationClip(tracks=[track1])
animation1 = AnimationAction(mixer=AnimationMixer(), clip=clip1, localRoot=meshes[11])

In [ ]:
track2 = VectorKeyframeTrack(name='.position', times=[0,1], values=[0.,0.,0., 0.,0.,300.])
clip2 = AnimationClip(tracks=[track1, track2])
animation2 = AnimationAction(mixer=AnimationMixer(), clip=clip2, localRoot=meshes[12])  

In [ ]:
rotation_null = Quaternion(axis=[0,0,1], degrees=0.)

In [ ]:
rotation_null = list(rotation_null.elements)

In [ ]:
rotation_z = Quaternion(axis=[0,0,1], degrees=90.)

In [ ]:
rotation_z = list(rotation_z.elements)

In [ ]:
rotation_null.extend(rotation_z)

In [ ]:
values = rotation_null

In [ ]:
track3 = QuaternionKeyframeTrack(name='.quaternion', times=[0,1], values=values)
clip3 = AnimationClip(tracks=[track3])
animation3 = AnimationAction(mixer=AnimationMixer(), clip=clip3, localRoot=meshes[12])  

In [ ]:
animation1.loop = 'LoopOnce'

In [ ]:
animation2.loop = 'LoopOnce'

In [ ]:
animations = [animation1, animation2]
for i in range(2):
    animations[i].play()

In [ ]:
scene2 = Scene(children=[meshes[1], meshes[2]])

In [ ]:
# [(name, type(getattr(meshes[0], name))) for name in dir(meshes[0])]

In [ ]:
# [(name,type(getattr(animation1,name))) for name in dir(animation1)]

In [ ]:
# [(name,type(getattr(track1,name))) for name in dir(track1)]

In [ ]:
camera_action

In [ ]:
# Everything below here contains tests, part of which don't work

In [ ]:
arm.positions['actuator']

In [ ]:
slide2.getAbsoluteLocation()

In [ ]:
# test on combining rotations 
r = transform.Rotation.from_rotvec([np.pi/2, 0, 0])
r.apply([1,2,3])

In [ ]:
armBreadboard.provenance()

In [ ]:
# reveal what units are used on the user side (external) and for storage/description (internal)
slide1.units()

In [ ]:
# slide1.store()

In [ ]:
slide1.extent

In [ ]:
slide1.loadStl()
slide1.extentFromStl(setExtent = True)

In [ ]:
# Create a new plot
figure = plt.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(slide1.stl.vectors))

cube = slide1.unityCube()
cube.vectors *= [20, 1, 30]
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(cube.vectors))


# Auto scale to the mesh size
scale = slide1.stl.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)
# doesn't work:
# mplot3d.art3d.rotate_axes(0, 0, 90, 'z')
# Show the plot to the screen
plt.show()

In [ ]:
cube.transform?

In [ ]:
slide3 = glassSlide(filename = Path('testSlide3.h5'), deleteExisting = True, loadFromFile = Path('testSlide.h5'))

In [ ]:
slide3._provenance

In [ ]:
slide1._provenance

In [ ]:
slide2.location = [3, 5, 8]

In [ ]:
slide1._provenance

In [ ]:
N = 1000
x, y, z = np.random.normal(0, 1, (3, N))

In [ ]:
fig=ipv.figure()
scatter = ipv.scatter(x, y, z, marker = 'sphere')
ipv.show()

In [ ]:
ds = vaex.example()
N = 2000 # for performance reasons we only do a subset
x, y, z, vx, vy, vz, Lz, E = [ds.columns[k][:N] for k in "x y z vx vy vz Lz E".split()]

ipv.clear()
quiver = ipv.quiver(np.array((1.1,2.2)), np.array((2.1, 3.1)), np.array((3.1, 4.1)), np.array((4.1, 5.1)), np.array((5.1, 6.1)), np.array((6, 7)), size=20)
ipv.show()

In [ ]:
slideRMesh = mesh.Mesh.from_file('ObjectClasses/STLs/Slide_rack_text.stl')
print(f'extent: {find_mins_maxs(slideRMesh)}')

In [ ]:
# Create a new plot
figure = plt.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
slideRMesh = mesh.Mesh.from_file('ObjectClasses/STLs/Slide_rack_text.stl')
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(slideRMesh.vectors))

# Auto scale to the mesh size
scale = slideRMesh.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)
# doesn't work:
# mplot3d.art3d.rotate_axes(0, 0, 90, 'z')
# Show the plot to the screen
plt.show()

In [ ]:
ipv.figure()
# we draw the tetrahedron
mesh = ipv.plot_surface(slideRMesh.x, slideRMesh.y, slideRMesh.z, color='orange')
# and also mark the vertices
ipv.xyzlim(np.min(find_mins_maxs(slideRMesh)), np.max(find_mins_maxs(slideRMesh)))
ipv.show()